In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
device=torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [4]:
model=models.vgg19(pretrained=True).features

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


In [5]:
print(model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [6]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        
        torch.autograd.set_detect_anomaly(True)
        self.chosen_features=['0','5','10','19','28']
        self.model=models.vgg19(pretrained=True).features[:29]
        
    def forward(self,x):
        features=[]
        
        for idx,layer in enumerate(self.model):
            x=layer(x)
            
            if str(idx) in self.chosen_features:
                features.append(x)
            
        return features
    

In [10]:
model=VGG().to(device).eval() #eval to freeze weights

In [12]:
def load_image(img_name):
    image=Image.open(img_name)
    image=image.resize((img_size,img_size))
    image = transform(image).unsqueeze(0)
    return image.to(device)

In [13]:
img_size=300

transform = transforms.Compose([transforms.ToTensor()])
        #transforms.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225))])

loader=transform

In [14]:
original_img=load_image("/content/gdrive/MyDrive/NST/saranga2.jfif")
style_img=load_image("/content/gdrive/MyDrive/NST/monet.jpeg")

In [15]:
#generated=torch.randn(original_img.shape,device=device,requires_grd=True)
generated=original_img.clone().requires_grad_(True)

In [16]:
total_steps=6000
lr=0.001
alpha=1
beta=0.01
optimizer=optim.Adam([generated],lr=lr)

In [ ]:
for step in tqdm(range(total_steps)):
        
        # Extract multiple(5) conv feature vectors
        target_features = model(generated)
        content_features = model(original_img)
        style_features = model(style_img)

        style_loss = 0
        content_loss = 0
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            # Compute content loss with target and content images
            content_loss += torch.mean((f1 - f2)**2)

            # Reshape convolutional feature maps
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)

            # Compute gram matrix
            G = torch.mm(f1, f1.t())
            A = torch.mm(f3, f3.t())

            # Compute style loss with target and style images
            style_loss += torch.mean((G - A)**2) 
        
        # Compute total loss, backprop and optimize
        total_loss = alpha*content_loss + beta * style_loss 
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        
        if step%200==0:
            print(total_loss)
            save_image(generated,"generated.png")

  0%|          | 2/6000 [00:00<20:59,  4.76it/s]

tensor(293026.7500, device='cuda:0', grad_fn=<AddBackward0>)


  3%|▎         | 202/6000 [00:32<16:04,  6.01it/s]

tensor(2130.5027, device='cuda:0', grad_fn=<AddBackward0>)


  7%|▋         | 402/6000 [01:05<15:49,  5.90it/s]

...

 10%|█         | 602/6000 [01:38<15:01,  5.99it/s]

tensor(791.9131, device='cuda:0', grad_fn=<AddBackward0>)


 13%|█▎        | 802/6000 [02:11<14:36,  5.93it/s]

tensor(628.5001, device='cuda:0', grad_fn=<AddBackward0>)


 15%|█▌        | 903/6000 [02:28<14:09,  6.00it/s]

In [18]:
generated.shape

torch.Size([1, 3, 300, 300])